In [4]:
from bs4 import BeautifulSoup
from deep_translator import GoogleTranslator
import re
from unidecode import unidecode
import string
import pandas as pd
import email

In [12]:
class DataPreProcessor():
    def __init__(self):
        self.translator = GoogleTranslator(source='zh-CN', target='en')

    def html_remover(self, doc):
        """
        Removes html tags in a given text
        Input: doc
        Output: String
        """
        txt = doc["pipe_text"]
        soup=BeautifulSoup(txt,'html.parser')
        a=soup.get_text()
        doc["pipe_text"] = a
        return doc

    # char limit 5000 currently 100000

    def illegal_char_remover(self, doc):
        """
        Removes some illegal characters that are not in unicode (represented in hex or bytes).
        """
        txt = doc["pipe_text"]
        txt_encoded = txt.encode("unicode_escape")
        txt_encoded_cleaned = re.sub(b'\\\\x[a-f0-9][a-f0-9]', b'', txt_encoded)
        txt_cleaned = txt_encoded_cleaned.decode("unicode_escape")
        doc["pipe_text"] = txt_cleaned
        return doc

    #Additions by Josiah

    def replace_tokens(self, doc):
        '''
        Replaces links, money, ip, numbers
        '''
        text = doc["pipe_text"]
        # Replace links with token [link]
        text = re.sub("http\S+", " [link] ", text)
        # Replace money with token [MONEY]
        text = re.sub(r"[$]\d+[.,]*\d*", " [money] ", text)
        text = re.sub(r"\d+[.,]*\d*[$]", " [money] ", text)
        # Replace ip addresses with token [ip]
        text = re.sub(r"^\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}$", " [ip] ", text)
        # Replace emails with token [email]
        text = re.sub(r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b", " [email] ", text)
        # Remove words with a number in the middle, like s8d or 7a8s8r. Does not remove office365 or 250k
        text = re.sub(r"\d*(([a-zA-Z]+)(\d+))+[a-zA-Z]+\d*", " ", text)
        # Replace numbers with a special token [NUM]
        text = re.sub(r"[\d]+", " [num] ", text)
        # Standardise whitespace
        text = re.sub('[\s]+'," ", text)
        # lowercase text
        text = text.lower()
        doc["pipe_text"] = text
        return doc
    
    def remove_nonenglish(self, doc):
        '''
        Removes characters from other languages
        '''
        text = doc["pipe_text"]
        text = re.sub(r'[^a-zA-Z0-9\]!"$%&\'()*+,./:;=#@?[\\^_`{|}~\-[\u00C0-\u01DA]+', " ", text) #Remove characters from other languages, except diacritics
        text = unidecode(text) # strip accents
        doc["pipe_text"] = text
        return doc
    
    def remove_punctuations(self, doc):
        text = doc["pipe_text"]
        text = text.translate(str.maketrans('', '', string.punctuation))
        doc["pipe_text"] = text
        return doc
        
    def join_spaces(self, text):
        '''
        Joins single letters that are separated by a single space, e.g w a n t  m o r e -> want more
        '''
        split = text.split(" ")
        ans = []
        temp = []
        for part in split:
            if len(part) == 1:
                temp.append(part)
            else:
                if len(temp)>0:
                    word = ''.join(temp)
                    ans.append(word)
                temp = []
                if len(part) != 0:
                    ans.append(part)
        word = ''.join(temp)
        ans.append(word)
        ans = " ".join(ans)
        return ans

    def clean_unicode(self, doc):
        '''
        Attempts to remove nonsensical characters and convert letterlike symbols back to letters.
        '''
        text = doc["pipe_text"]
        words = []
        text = str(text)
        text = text.replace("\n", "  ")
        text = text.replace(chr(160)," ") # Replace weird space
        text = re.sub(r"[\u2800-\u28ff]+", "", text)
        text = self.join_spaces(text)
        for word in text.split(" "):
            if len(word) == 0:
                continue
            if unidecode(word) == "":
                continue
            code = [ord(char) for char in word]
            if max(code)>65535: # Outside Basic multilangual plane
                word = unidecode(word)
            words.append(word)
        text = " ".join(words)
        doc["pipe_text"] = text
        return doc

    # def detect_lang(self, doc):
    #     '''
    #     Tries to detect the language of text, reducing number of text that has to be passed through translator.
    #     Note that it is not accurate at detecting the actual language, just weeds out English text.
    #     '''
    #     text = doc["pipe_text"]
    #     text = re.sub("[link]", "", text)
    #     text = re.sub(r"[!'#$%&\"()*+,-./:;<=>?@[\\\]^_`{|}~]+", " ", text)
    #     text = re.sub(r"[0-9]+", " ", text)
    #     if len(text) == 0:
    #         doc["lang"] = None
    #     try:
    #         lang = detect(text)
    #     except:
    #         text = unidecode(text)
    #         try:
    #             lang = detect(text)
    #         except:
    #             print(text)
    #             doc["lang"] = None
    #             return doc
    #     doc["lang"] = lang
    #     return doc

    def translate(self, doc, char_limit=50000):
        """
        Can only deal with one string (not a list)
        """
        text = doc["pipe_text"]
        if doc.get("lang") == "en":
            doc["translated"] = text
            doc["pipe_text"] = text
            return doc
        # Limit the translation to 50000 chars. If larger, dont translate.
        if len(text) <= char_limit:

            if len(text) <= 1000:
                try:
                    translated = self.translator.translate(text)
                except Exception as e:
                    print(e)
                    translated = text
                if translated==None:
                    translated = ""
                doc["translated"] = translated
                doc["pipe_text"] = doc["translated"]
                return doc
            
            # Split text into chunks that the translation engine can handle. We limit this to 4000 char as of now
            words = doc["pipe_text"].split(" ")
            chunks = []
            char_count = 0
            temp_chunk = []
            for word in words:
                char_count += (len(word) + 1)
                if char_count <= 1000:
                    temp_chunk.append(word)
                else:
                    chunks.append(" ".join(temp_chunk))
                    char_count = len(word)
                    temp_chunk = [word]

            if len(temp_chunk) > 0:
                chunks.append(" ".join(temp_chunk))

            new_pipe_text = []
            for chunk in chunks:
                if len(chunk) > 1000:
                    translated_text = chunk
                else:
                    try:
                        translated_text = self.translator.translate(chunk)
                    except Exception as e:
                        print(e)
                        translated_text = chunk
                    if not translated_text:
                        translated_text = ''
                new_pipe_text.append(translated_text)

            full_translated = ' '.join(new_pipe_text)
        else:
            full_translated = text
        if full_translated == None:
            full_translated = ""
        doc["translated"] = full_translated
        doc["pipe_text"] = full_translated
        return doc
    
 
    def preprocess(self, text, lang, first_pipe, second_pipe, doc_char_limit=5000):
        """
        The returned doc might have the following fields, depending on the pipes
        - original_text
        - translated
        - sents
        - pipes is a list of lists of various pipe components 
            E.g. [[html_remover, illegal_char_remover, translator], [url_remover, consec_newliine_remover]]
        """

        # Limit the number of characters if not there will be a memory error
        doc = {
            "original_text": text,
            "pipe_text": text,
            "pipe_outputs": [],
            "lang": lang
        }

        # Sentencizer has to be the last pipe.
        pipe_component_to_func = {
            "html_remover": self.html_remover,
            "illegal_char_remover": self.illegal_char_remover,
            "translator": self.translate,
            "clean_unicode": self.clean_unicode,
        }
        for pipe_component in first_pipe:
            doc = pipe_component_to_func[pipe_component](doc)
        if not doc.get("translated"):
            doc["translated"] = doc["pipe_text"]
        # Truncate pipe text and then do the remaining operations
        doc["first_pipe_text"] = doc["pipe_text"]
        doc["pipe_text"] = doc["pipe_text"][:doc_char_limit]
        second_pipe_component_to_func = {
            "remove_nonenglish": self.remove_nonenglish,
            "remove_punctuation": self.remove_punctuations,
            "replace_tokens": self.replace_tokens,
        }

        for pipe_component in second_pipe:
            doc = second_pipe_component_to_func[pipe_component](doc)
        doc["second_pipe_text"] = doc["pipe_text"]
        doc.pop("pipe_text")
        return doc

In [5]:
ch_emails_raw = pd.read_pickle("Data/ch_emails_raw.pkl")

In [7]:
temp = ch_emails_raw.iloc[:100]

In [13]:
temp_message = temp.iloc[0].message
lang = "zh-CN"
first_preprocess_pipe = ["html_remover", "clean_unicode", "translator"]
second_preprocess_pipe = ["remove_nonenglish", "replace_tokens", "remove_punctuation"]

In [14]:
data_preprocessor = DataPreProcessor()

In [16]:
data_preprocessor.preprocess(temp_message[0], lang, first_preprocess_pipe, second_preprocess_pipe)

{'original_text': '讲的是孔子后人的故事。一个老领导回到家乡，跟儿子感情不和，跟贪财的孙子孔为本和睦。\n老领导的弟弟魏宗万是赶马车的。\n有个洋妞大概是考察民俗的，在他们家过年。\n孔为本总想出国，被爷爷教育了。\n最后，一家人基本和解。\n顺便问另一类电影，北京青年电影制片厂的。中越战背景。一军人被介绍了一个对象，去相亲。女方是军队医院的护士，犹豫不决，总是在回忆战场上负伤的男友，好像还没死。最后\n男方表示理解，归队了。\n',
 'pipe_outputs': [],
 'lang': 'zh-CN',
 'translated': "It tells the story of the descendants of Confucius. An old leader returned to his hometown and was at odds with his son and at peace with his money-grubbing grandson Kong Weiben. Wei Zongwan, the old leader's younger brother, drives the carriage. There was a foreign girl who was probably investigating folk customs and was celebrating the New Year at their house. Kong Weiben always wanted to go abroad and was educated by his grandfather. In the end, the family basically reconciled. By the way, I want to ask about another type of film, produced by Beijing Youth Film Studio. Background of the Sino-Vietnam War. A soldier was introduced to a date and went on a blind date. The woman is a nurse in a military hospi

In [ ]:
if lang != "en":
        first_preprocess_pipe = ["html_remover", "clean_unicode", "detect_lang", "translator"]
        second_preprocess_pipe = ["remove_nonenglish", "replace_tokens", "remove_punctuation"]
    else:
        first_preprocess_pipe = ["html_remover", "clean_unicode"]
        second_preprocess_pipe = ["remove_nonenglish", "replace_tokens", "remove_punctuation"]

In [17]:
ch_emails_raw["multipart"] = ch_emails_raw.message.apply(lambda x: len(x)>1)
ch_emails_raw.head()

,class,subpath,subject,email_to,email_from,message,multipart
1,ham,../data/000/001,● 问一部魏宗万的电影名称,shi@ccert.edu.cn,"""pan"" <pan@jdl.ac.cn>",[讲的是孔子后人的故事。一个老领导回到家乡，跟儿子感情不和，跟贪财的孙子孔为本和睦。\n老领...,False
2,spam,../data/000/002,公司业务.代开发票！,xing@ccert.edu.cn,张海南 <jian@163.con>,[尊敬的贵公司(财务/经理)负责人您好！ \n 我是深圳金海实业有限公司（广...,False
3,spam,../data/000/003,低点代开发票!,ling@ccert.edu.cn,代开发票 <pan@12.com>,[贵公司负责人(经理/财务）您好： \n 深圳市华龙公司受多家公司委托向外低点代开部分...,False
4,spam,../data/000/004,一边上网冲浪，一边赚钱，何乐而不为？,tang@ccert.edu.cn,"""mei"" <mei@dghhkjk.com>",[这是一封HTML格式信件！\n\n----------------------------...,False
5,spam,../data/000/005,优惠代开各种发票!!,yuan@ccert.edu.cn,"ke@163.com"" <chunyang-sz@163.com>",[\nTO：贵公司经理、财务\n\n \n 您好！ \n 深圳市春洋贸易...,False


In [18]:
ch_emails_raw[ch_emails_raw.multipart]

,class,subpath,subject,email_to,email_from,message,multipart


In [35]:
en_emails_raw = pd.read_pickle("Data/en_emails_raw.pkl")

In [36]:
en_emails_raw["multipart"] = en_emails_raw.message.apply(lambda x: len(x)>1)
en_emails_raw[en_emails_raw.multipart].iloc[20:30]

,class,subpath,subject,email_to,email_from,message,multipart
56,spam,../data/000/056,we will not disappoint you,gina@groucho.cs.psu.edu,"""Mr Kali Lydon"" <xmfulrbs@1internet.us>",[! phobic in creamy it kaolin try midget ! idi...,True
59,spam,../data/000/059,Three Steps to the Software You Need at the Pr...,<angelina@groucho.cs.psu.edu>,"""Martin Perez"" <proftechn@queenschiropractor.com>",[\nSpecial Offer\nAdobe Video Collection\nAdob...,True
61,ham,../data/000/061,Quadra Utilities,macgifts@mac.archive.umich.edu,Beth Corbo <corbo@lclark.edu>,[ Here are the utilities that I found for dea...,True
63,spam,../data/000/063,American Fat Reducing,maude@groucho.cse.psu.edu,"""Hambrick"" <klwjqnytbvffh@stephouse.net>",[but jejunum in gyro it's satanic on judge but...,True
64,spam,../data/000/064,Re: your AMBtjEN,bobbie@groucho.cse.psu.edu,"""Jocelin Brundige"" <brundigeu@techno-logy.nl>",[Hi\n=20\nL\nC\nV\nP\nV\nX\nA\ne\nI\nA\nr\nI\n...,True
68,spam,../data/000/068,We never repent of having eaten too little,stephanie@groucho.cse.psu.edu,"""Carmela Burger"" <krgenihybkm@lingo.com>",[or indict or rhetoric in galaxy try stapleton...,True
80,spam,../data/000/080,We never repent of having eaten too little,"geneva@groucho.cse.psu.edu, paul@groucho.cse.p...","""Shirley Ledford"" <ffacqfhbsahy@lingo.com>",[not bellflower on cromwellian a embroider see...,True
84,spam,../data/000/084,Re: your AMBtjEN,bobbie@groucho.cse.psu.edu,"""Jocelin Brundige"" <brundigeu@techno-logy.nl>",[Hi\n=20\nL\nC\nV\nP\nV\nX\nA\ne\nI\nA\nr\nI\n...,True
85,spam,../data/000/085,American Fat Reducing,maude@groucho.cse.psu.edu,"""Hambrick"" <klwjqnytbvffh@stephouse.net>",[but jejunum in gyro it's satanic on judge but...,True
86,spam,../data/000/086,We never repent of having eaten too little,stephanie@groucho.cse.psu.edu,"""Carmela Burger"" <krgenihybkm@lingo.com>",[or indict or rhetoric in galaxy try stapleton...,True


In [37]:
temp = en_emails_raw[en_emails_raw.multipart]
temp_message = temp[temp["class"]=="ham"].iloc[0]["message"]

In [38]:
temp_message

['  Here are the utilities that I found for dealing with\nsetting the amount of time the Mac waits for the\ninternal hard drive to spin-up and some notes\non SCSI from Micronet.\n  Enjoy!\n\n  Beth Corbo (corbo@lclark.edu)\n\n\n',
 'Macintosh SCSI Termination\nby the Engineering Department of\nMicroNet Technology, Inc.\n \nDated:  11/1/90\n \n \nWhat is Termination?\n \nTermination involves the placement of impedance matching circuits on the bus.\nIn the case of SCSI, these circuits are typically placed at each end of the\nSCSI Bus (cable). These circuits are known as "Thevenin Terminators" and are\nsimple resistor networks in the case of SCSI.  These resistor circuits are\ncomposed of a 220 ohm resistor connected to Vcc and the signal node, while a\n330 ohm resistor is connected to ground and the signal node.  One of these\nresistor pairs is connected to each of the 18 active signal lines.  The Vcc (+5\nnominal) for the terminators is typically provided by line 26 on the SCSI bus\ncab